In [1]:
import os
from time import time

import openai
from dotenv import load_dotenv, find_dotenv
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(temperature=0)

prompt = ChatPromptTemplate.from_template(
    'Classify the text based on the Common European Framework of Reference '
    'for Languages (CEFR). Give a single value: {text}',
)
chain = LLMChain(llm=llm, prompt=prompt)

texts = [
    {'text': 'Hallo, ich bin 25 Jahre alt.'},
    {'text': 'Wie geht es dir?'},
    {'text': 'In meiner Freizeit, spiele ich gerne Fussball.'}
]

start = time()
res_a = chain.apply(texts)
print(res_a)
print(f"apply time taken: {time() - start:.2f} seconds")
print()

start = time()
res_aa = await chain.aapply(texts)
print(res_aa)
print(f"aapply time taken: {time() - start:.2f} seconds")

[{'text': 'Based on the given text "Hallo, ich bin 25 Jahre alt," it can be classified as CEFR level A1.'}, {'text': 'A2'}, {'text': 'A2'}]
apply time taken: 2.79 seconds

[{'text': 'Based on the given text "Hallo, ich bin 25 Jahre alt," it can be classified as CEFR level A1.'}, {'text': 'A2'}, {'text': 'A2'}]
aapply time taken: 2.05 seconds


In [22]:
import openai
import os
from time import time
from pprint import pprint

from pprint import pprint

import pandas as pd
from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
)
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv('OPENAI_API_KEY')

In [4]:
df = pd.read_csv('../data/product_reviews.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


# Langy

### Classify Level of Text and Provide Reason

In [5]:
llm = ChatOpenAI(temperature=0)

cefr_template ="""Classify the text based on the Common European Framework of Reference
for Languages (CEFR), provide a reason for your answer.

Text: {text}

{format_instructions}
 """

prompt = ChatPromptTemplate.from_template(cefr_template)

chain = LLMChain(llm=llm, prompt=prompt)

# Define output schema
level_schema = ResponseSchema(name="level",
                             description="The CEFR level of the text, e.g. A1, B2, etc.")
reason_schema = ResponseSchema(name="reason",
                                    description="The reason for the classification")

response_schemas = [level_schema, reason_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

text = """Hallo, ich heisse Adam. Ich habe 25 Jahre alt. Ich wohne in England seit 15 Jahren aber ich wuerde gerne irgendwo anders wohnen. Ich liebe es zu reisen. Meiner Meinung nach, ist man wirklich am leben, wenn man reist."""

# messages = prompt.format(text=text, format_instructions=format_instructions)

# messages = [
#     {'text': text, 'format_instructions': format_instructions}
# ]
# response = chain.apply(messages)

messages = {'text': text, 'format_instructions': format_instructions}
response = chain.run(messages)

result = output_parser.parse(response)
print(result)
type(result)
# response[0]['text']

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"level": string  // The CEFR level of the text, e.g. A1, B2, etc.
	"reason": string  // The reason for the classification
}
```
{'level': 'A2', 'reason': 'The text demonstrates a basic understanding of the language with simple sentence structures and limited vocabulary. The use of present tense and basic expressions indicate a beginner level of proficiency.'}


dict

In [23]:
text = 'Hallo, ich heisse Adam. Ich habe 25 Jahre alt und fuenf jungere Schwester.'

llm = ChatOpenAI(
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

template_reason_level ="""Classify the text based on the Common European Framework of Reference
for Languages (CEFR), provide detailed reasons for your answer.

Text: {text}

{format_instructions}
 """

class ReasonLevel(BaseModel):
    reason: str = Field(description="Detailed reasons for the classification")
    level: str = Field(description="The CEFR level of the text, e.g. A1, B2, etc.")

parser_reason_level = PydanticOutputParser(pydantic_object=ReasonLevel)

prompt_template_reason_level = ChatPromptTemplate(
    messages=[HumanMessagePromptTemplate.from_template(template_reason_level)],
    input_variables=['text'],
    partial_variables={'format_instructions': parser_reason_level.get_format_instructions()}
)

chain_reason_level = LLMChain(
    llm=llm,
    prompt=prompt_template_reason_level,
    output_key='reason_level'
)

output_reason_level = chain_reason_level({'text': text})
results_reason_level = parser_reason_level.parse(output_reason_level['reason_level'])
results_reason_level

{"reason": "The text can be classified as A1 level based on the following reasons:\n\n1. Vocabulary: The text uses basic vocabulary such as 'Hallo' (Hello), 'ich' (I), 'heisse' (am called), 'alt' (old), 'und' (and), 'jungere' (younger), and 'Schwester' (sister). These are common words that are typically learned at the beginner level.\n\n2. Sentence Structure: The text consists of simple sentences with basic grammatical structures. For example, 'Ich habe 25 Jahre alt' (I am 25 years old) and 'fuenf jungere Schwester' (five younger sisters). The sentence structure is straightforward and does not involve complex grammar.\n\n3. Lack of Complex Expressions: The text does not contain any complex expressions or idiomatic phrases. It uses simple and direct language to convey basic information about the speaker.\n\nBased on these factors, the text aligns with the A1 level of the CEFR.", "level": "A1"}

ReasonLevel(reason="The text can be classified as A1 level based on the following reasons:\n\n1. Vocabulary: The text uses basic vocabulary such as 'Hallo' (Hello), 'ich' (I), 'heisse' (am called), 'alt' (old), 'und' (and), 'jungere' (younger), and 'Schwester' (sister). These are common words that are typically learned at the beginner level.\n\n2. Sentence Structure: The text consists of simple sentences with basic grammatical structures. For example, 'Ich habe 25 Jahre alt' (I am 25 years old) and 'fuenf jungere Schwester' (five younger sisters). The sentence structure is straightforward and does not involve complex grammar.\n\n3. Lack of Complex Expressions: The text does not contain any complex expressions or idiomatic phrases. It uses simple and direct language to convey basic information about the speaker.\n\nBased on these factors, the text aligns with the A1 level of the CEFR.", level='A1')

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationTokenBufferMemory

llm = ChatOpenAI(temperature=0.0, streaming=True)
DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is helping the human improve their foreign language writing skills. The human provides texts written in a foreign language and the AI corrects the spelling and grammar of the texts and provides detailed reasons for each correction.

The AI keeps in in mind spelling, grammar, naturalness (how much it sounds like a native speaker), correct capitalisation,
correct placement of commas or other punctuation and anything else necessary for correct writing.

The AI only provides corrections for words/phrases that have changed. If the original text is the same as the corrected text, then the AI does not provide a correction.

The AI knows that each sentence may contain multiple errors and provides corrections for all errors in the sentence.
It also knows that some sentences will not contain any errors and does not provide corrections for those sentences.

If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI: Let's think step by step"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=DEFAULT_TEMPLATE)

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=4000)
# system_prompt = """I want you to help me improve my foreign language writing skills.
# I will give you texts in a foreign language. Please correct the spelling and grammar
# of them.
#
# Keep in mind spelling,
# grammar, naturalness (how much it sounds like a native speaker), correct capitalisation,
# correct placement of commas or other punctuation and anything else necessary for correct
# writing.
#
# Provide a detailed reason for each correction.
#
# Do not do anything else other than correct the text I've just passed in and providing explanations.
#
# Please reply with "Ok!" to show you have understood. The next message I send after that will be
# the first text you need to correct.
# """
#
# memory.save_context(
#     {'input': system_prompt},
#     {'output': 'Ok!'}
# )

input_1 = 'Hallo, ich heisse Adam. Ich habe 25 Jahre alt.'
output_1 = """Let's think step by step
Corrected text: Ich heiße Adam. Ich bin 25 Jahre alt.

Reasons:
1. Corrected spelling of 'heisse' to 'heiße' because 'heiße' is the correct spelling in German
2. Corrected 'alt' to 'bin' because 'bin' is the correct verb to use when stating one's age in German."""
memory.save_context(
    {'input': input_1},
     {'output': output_1}
)

input_2 = 'Ich bin 25 Jahre alt'
output_2 = """Let's think step by step
Corrected text: Ich bin 25 Jahre alt.

Reasons:
1. Added full stop to the end of the sentence because it is a complete sentence."""
memory.save_context(
    {'input': input_2},
     {'output': output_2}
)

input_3 = "Ich habe eine Katze. Sie ist schwarz und klein."
output_3 = """Let's think step by step
Corrected text: Ich habe eine Katze. Sie ist schwarz und klein.

Reasons:
1. No corrections needed. The text is grammatically correct and natural."""

memory.save_context(
    {'input': input_3},
     {'output': output_3}
)

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True,
    prompt=PROMPT,
)

conversation.predict(input='Ich wohne auf England fuer 15 Jahren.')

tom = """„Tom! Tom! Wo sind du?“
Keine Antwort.
„Wo ist dem bloß? Wenn ich ihm finde,
werde ich...“
Tante Polly schaut unter das Bett. Dann
öffnet sie die Haustür und schaut in den
Garten."""
x = conversation.predict(input=tom)

eltern = """Eltern klagen, dass ihre Kinder ihnen nicht gehorchen, Streit den Alltag bestimmt. Lehrer klagen über freche bis gewalttätige Schüler und, das fehlenden Interesse der Eltern an der Schule nach. Sie fühlen sich mit der Aufgabe überfordert, gleichzeitig Erziehungs- und Bildungsarbeit leisten zu mussen."""

x = conversation.predict(input=eltern)
x



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is helping the human improve their foreign language writing skills. The human provides texts written in a foreign language and the AI corrects the spelling and grammar of the texts and provides detailed reasons for each correction.

The AI keeps in in mind spelling, grammar, naturalness (how much it sounds like a native speaker), correct capitalisation,
correct placement of commas or other punctuation and anything else necessary for correct writing.

The AI only provides corrections for words/phrases that have changed. If the original text is the same as the corrected text, then the AI does not provide a correction.

The AI knows that each sentence may contain multiple errors and provides corrections for all errors in the sentence.
It also knows that some sentences will not contain any errors and does not provide corrections for those sentences

"Corrected text: Eltern klagen, dass ihre Kinder ihnen nicht gehorchen, Streit den Alltag bestimmt. Lehrer klagen über freche bis gewalttätige Schüler und das fehlende Interesse der Eltern an der Schule. Sie fühlen sich mit der Aufgabe überfordert, gleichzeitig Erziehungs- und Bildungsarbeit leisten zu müssen.\n\nReasons:\n1. Removed the comma after 'Schüler' because it is not necessary.\n2. Corrected 'das fehlenden' to 'das fehlende' because 'fehlende' is the correct form of the adjective to use with 'Interesse' (interest).\n3. Removed 'nach' after 'Schule' because it is not necessary.\n4. Corrected 'mussen' to 'müssen' because 'müssen' is the correct form of the verb to use in this sentence."

In [13]:
x

'AI: First, let\'s correct the sentence "Wo sind du?" to "Wo bist du?" because "bist" is the correct verb form to use with the pronoun "du" in German.\n\nNext, let\'s correct the sentence "Wo ist dem bloß?" to "Wo ist er bloß?" because "er" is the correct pronoun to use when referring to a male person.\n\nFinally, let\'s correct the sentence "Tante Polly schautet" to "Tante Polly schaut" because "schautet" is not a correct verb form in German. The correct form is "schaut."\n\nPutting it all together, the corrected text is:\n"Tom! Tom! Wo bist du?"\nKeine Antwort.\n"Wo ist er bloß? Wenn ich ihn finde,\nwerde ich..."\nTante Polly schaut unter das Bett. Dann\nöffnet sie die Haustür und schaut in den\nGarten.'

In [12]:
template_correct = """"I want you to help me improve my foreign language writing skills.
Here is a text I've written in a foreign language. Please correct the spelling and grammar
of the text.

Keep in mind spelling,
grammar, naturalness (how much it sounds like a native speaker), correct capitalisation,
correct placement of commas or other punctuation and anything else necessary for correct
writing. Provide a detailed reason for each correction.

Text: {text}
"""
template_correct_with_format = template_correct + """\n\n{format_instructions}"""

class CorrectedText(BaseModel):
    corrected_text: str = Field(description="The corrected text")
    reasons: List[str] = Field(description="Detailed reasons for each correction")

parser_corrected_text = PydanticOutputParser(pydantic_object=CorrectedText)

prompt_template_correct = ChatPromptTemplate(
    messages=[HumanMessagePromptTemplate.from_template(template_correct)],
    input_variables=['text'],
    partial_variables={'format_instructions': parser_corrected_text.get_format_instructions()}
)

chain_correct = LLMChain(
    llm=llm,
    prompt=prompt_template_correct,
    output_key='corrected_text'
)

output_correct = chain_correct({'text': text})
results_correct = parser_corrected_text.parse(output_correct['corrected_text'])
results_correct

output_correct

{'text': 'Hallo, ich heisse Adam. Ich habe 25 Jahre alt und fuenf jungere Schwester.',
 'corrected_text': '{"corrected_text": "Hallo, ich heiße Adam. Ich bin 25 Jahre alt und habe fünf jüngere Schwestern.", "reasons": ["Corrected spelling of \'heisse\' to \'heiße\' because \'heiße\' is the correct spelling in German.", "Corrected \'alt\' to \'bin\' because \'bin\' is the correct verb to use when stating one\'s age in German.", "Corrected \'jungere\' to \'jüngere\' because \'jüngere\' is the correct spelling of \'younger\' in German.", "Changed \'Schwester\' to \'Schwestern\' because \'Schwestern\' is the plural form of \'Schwester\'."] }'}

In [29]:
output_correct

{'text': 'Hallo, ich heisse Adam. Ich habe 25 Jahre alt und fuenf jungere Schwester.',
 'corrected_text': '{"corrected_text": "Hallo, ich heiße Adam. Ich bin 25 Jahre alt und habe fünf jüngere Schwestern.", "reasons": ["Corrected spelling of \'heisse\' to \'heiße\' because \'heiße\' is the correct spelling in German.", "Corrected \'alt\' to \'bin\' because \'bin\' is the correct verb to use when stating one\'s age in German.", "Corrected \'fuenf\' to \'fünf\' because \'fünf\' is the correct spelling of the number \'five\' in German.", "Corrected \'Schwester\' to \'Schwestern\' because \'Schwestern\' is the correct plural form of \'Schwester\' in German."]}'}

In [11]:
template_check = """Here is some input text in a foreign language, a corrected
version of the text and detailed reasons for the corrections.

For each reason, check if that is actually what has happened in the text.

Check the corrected text and reasons for each correction.
Do all the reasons make sense? Are the reasons themselves correct?
If not, please update the list of reasons and provide a reason for each update.

Each section is delimited by ####

Input text: ####{text}####
Corrected text: ####{corrected_text}####
Reasons: ####{reasons}####
"""

template_check = HumanMessagePromptTemplate.from_template(template_check)
example_human = template_check.format_messages(**{
    'text': 'Ich heisse Adam. Ich habe 25 Jahre alt.',
    'corrected_text': 'Ich heiße Adam. Ich bin 25 Jahre alt.',
    'reasons': ["Corrected spelling of 'heisse' to 'heiße' because 'heiße' is the correct spelling in German.", "Corrected 'alt' to 'bin' because 'bin' is the correct verb to use when stating one's age in German."]
})[0]
example_ai = AIMessagePromptTemplate.from_template(
    '''"reasons": ["Corrected spelling of 'heisse' to 'heiße' because 'ss' can be combined to
     'ß' in German.", "Corrected 'habe' to 'bin' because 'bin' is the correct verb to use when stating one's age in German."]'''
)

prompt_template_check = ChatPromptTemplate(
    messages=[example_human, example_ai, template_check],
    input_variables=['text', 'corrected_text', 'reasons'],
    # partial_variables={'format_instructions': parser_corrected_text.get_format_instructions()}
)

chain_check = LLMChain(
    llm=llm,
    prompt=prompt_template_check,
    output_key='check'
)
chain_check({
    'text': text,
    'corrected_text': results_correct.corrected_text,
    'reasons': results_correct.reasons
})

{'text': 'Hallo, ich heisse Adam. Ich habe 25 Jahre alt und fuenf jungere Schwester.',
 'corrected_text': 'Hallo, ich heiße Adam. Ich bin 25 Jahre alt und habe fünf jüngere Schwestern.',
 'reasons': ["Corrected spelling of 'heisse' to 'heiße' because 'heiße' is the correct spelling in German.",
  "Corrected 'alt' to 'bin' because 'bin' is the correct verb to use when stating one's age in German.",
  "Corrected 'fuenf' to 'fünf' because 'fünf' is the correct spelling of the number 'five' in German.",
  "Corrected 'Schwester' to 'Schwestern' because 'Schwestern' is the correct plural form of 'Schwester' in German."],
 'check': '"reasons": ["Corrected spelling of \'heisse\' to \'heiße\' because \'ss\' can be combined to\n     \'ß\' in German.", "Corrected \'habe\' to \'bin\' because \'bin\' is the correct verb to use when stating one\'s age in German.", "Corrected spelling of \'fuenf\' to \'fünf\' because \'fünf\' is the correct spelling of the number \'five\' in German.", "Corrected \'Sc

In [31]:
example_human

[HumanMessage(content='Here is some input text in a foreign language, a corrected\nversion of the text and detailed reasons for the corrections.\n\nFor each reason, check if that is actually what has happened in the text.\n\nCheck the corrected text and reasons for each correction.\nDo all the reasons make sense? Are the reasons themselves correct?\nIf not, please update the list of reasons and provide a reason for each update.\n\nEach section is delimited by ####\n\nInput text: ####Ich heisse Adam. Ich habe 25 Jahre alt.####\nCorrected text: ####Ich heiße Adam. Ich bin 25 Jahre alt.####\nReasons: ####["Corrected spelling of \'heisse\' to \'heiße\' because \'heiße\' is the correct spelling in German.", "Corrected \'alt\' to \'bin\' because \'bin\' is the correct verb to use when stating one\'s age in German."]####\n', additional_kwargs={}, example=False)]

In [32]:
example_ai

AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='"reasons": ["Corrected spelling of \'heisse\' to \'heiße\' because \'ss\' can be combined to\n     \'ß\' in German.", "Corrected \'habe\' to \'bin\' because \'bin\' is the correct verb to use when stating one\'s age in German."]', template_format='f-string', validate_template=True), additional_kwargs={})

In [25]:
x = HumanMessagePromptTemplate.from_template(template_check)
x.format_messages(**{
    'text': text,
    'corrected_text': results_correct.corrected_text,
    'reasons': results_correct.reasons
})

[HumanMessage(content='Here is some input text in a foreign language, a corrected\nversion of the text and detailed reasons for the corrections.\n\nCheck the corrected text and reasons for each correction.\nDo all the reasons make sense? Are the reasons themselves correct?\nIf not, please update the list of reasons and provide a reason for each update.\n\nEach section is delimited by ####\n\nInput text: ####Hallo, ich heisse Adam. Ich habe 25 Jahre alt und fuenf jungere Schwester.####\nCorrected text: ####Hallo, ich heiße Adam. Ich bin 25 Jahre alt und habe fünf jüngere Schwestern.####\nReasons: ####["Corrected spelling of \'heisse\' to \'heiße\' because \'heiße\' is the correct spelling in German.", "Corrected \'alt\' to \'bin\' because \'bin\' is the correct verb to use when stating one\'s age in German.", "Corrected \'fuenf\' to \'fünf\' because \'fünf\' is the correct spelling of the number \'five\' in German.", "Corrected \'Schwester\' to \'Schwestern\' because \'Schwestern\' is t

In [25]:
parser_reason_level.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"reason": {"title": "Reason", "description": "The reason for the classification", "type": "string"}, "level": {"title": "Level", "description": "The CEFR level of the text, e.g. A1, B2, etc.", "type": "string"}}, "required": ["reason", "level"]}\n```'

'A1'

### Correct Grammar and Spelling

In [32]:
llm = ChatOpenAI(temperature=0)

gram_spell_template ="""Correct the grammar and spelling of the text. Provide a reason for
each correction.

Text: {text}

{format_instructions}
 """

prompt = ChatPromptTemplate.from_template(gram_spell_template)

chain = LLMChain(llm=llm, prompt=prompt)

# Define output schema
corrected_schema = ResponseSchema(name="corrected_text",
                                    description="The corrected text")
reasons_schema = ResponseSchema(name="reasons",
                                description="The reasons for the corrections")


response_schemas = [corrected_schema, reasons_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

text = """Hallo, ich heisse Adam. Ich habe 25 Jahre alt. Ich wohne in England seit 15 Jahren aber ich wuerde gerne irgendwo anders wohnen. Ich liebe es zu reisen. Meiner Meinung nach, ist man wirklich am leben, wenn man reist."""

messages = {
    'text': text,
    'format_instructions': format_instructions
}
response = chain.run(messages)
# response

result = output_parser.parse(response)
print(result)
type(result)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"corrected_text": string  // The corrected text
	"reasons": string  // The reasons for the corrections
}
```
{'corrected_text': 'Hallo, ich heiße Adam. Ich bin 25 Jahre alt. Ich wohne seit 15 Jahren in England, aber ich würde gerne irgendwo anders wohnen. Ich liebe es zu reisen. Meiner Meinung nach ist man wirklich am Leben, wenn man reist.', 'reasons': "1. Corrected the spelling of 'heisse' to 'heiße' because it should have an 'ß' instead of 'ss'.\n2. Changed 'habe' to 'bin' because '25 Jahre alt' is a state of being, not possession.\n3. Added a comma after 'England' to separate the clauses.\n4. Removed the space before 'wuerde' to correct the spelling.\n5. Removed the space before 'Meiner' to correct the sentence structure.\n6. Removed the comma after 'nach' to improve the flow of the sentence."}


dict

In [5]:
texts = [
    {'text': 'Hallo, ich bin 25 Jahre alt.'},
    {'text': 'Wie geht es dir?'},
    {'text': 'In meiner Freizeit spiele ich gerne Fussball.'}
]
start = time()
res_a = chain.apply(texts)
print(res_a)
print(f"apply time taken: {time() - start:.2f} seconds")
print()

start = time()
res_aa = await chain.aapply(texts)
print(res_aa)
print(f"aapply time taken: {time() - start:.2f} seconds")

[{'text': 'Based on the provided text "Hallo, ich bin 25 Jahre alt," it can be classified as A1 level according to the Common European Framework of Reference for Languages (CEFR). \n\nThe reason for this classification is that the text consists of simple and basic vocabulary and sentence structures. It includes a greeting ("Hallo") and a simple statement about the person\'s age ("ich bin 25 Jahre alt"). A1 level is the beginner level, where learners can understand and use familiar everyday expressions and basic phrases to communicate simple and immediate needs.'}, {'text': 'According to the Common European Framework of Reference for Languages (CEFR), the text "Wie geht es dir?" can be classified as A1 level. \n\nThe reason for this classification is that the text consists of a simple question in German, asking "How are you?" This type of question is commonly taught at the beginner level, where learners are introduced to basic greetings and expressions. At the A1 level, learners are exp

In [11]:
chain.predict(a=texts[0])

ValueError: Missing some input keys: {'text'}

# LLMChain

In [14]:
# High temp -> more creative names
llm = ChatOpenAI(temperature=0.9)
prompt = ChatPromptTemplate.from_template(
    'What is the best name to describe a company that sells {product}?',
)
chain = LLMChain(llm=llm, prompt=prompt)

for product in df.Product.iloc[:5]:
    company_name_idea = chain.run(product)
    print(f"Product: {product}, Generated name: {company_name_idea}")

Product: Queen Size Sheet Set, Generated name: "Royal Dreams Linens" or "Regal Comfort Bedding"
Product: Waterproof Phone Pouch, Generated name: Aquashield
Product: Luxury Air Mattress, Generated name: 1. Opulent Rest
2. LuxeSlumber
3. Supreme Sleep
4. Elite Air Mattress Co.
5. DreamSerenity
6. LuxAir Mattress
7. PlushDreams
8. PrestigeSleep
9. RegalAir Mattress
10. SereneSlumber
Product: Pillows Insert, Generated name: The Cushion Corner
Product: Milk Frother Handheld
, Generated name: "Milk Frother Plus" or "FrothMaster"


In [20]:
# You can also generate multiple results at once like this
products = list(df.Product.iloc[:5])
products = [{'product': product} for product in products]
chain.generate(products)

LLMResult(generations=[[ChatGeneration(text='RoyalComfort Beddings', generation_info=None, message=AIMessage(content='RoyalComfort Beddings', additional_kwargs={}, example=False))], [ChatGeneration(text='AquaGuard Phone Accessories', generation_info=None, message=AIMessage(content='AquaGuard Phone Accessories', additional_kwargs={}, example=False))], [ChatGeneration(text='OpulentRest', generation_info=None, message=AIMessage(content='OpulentRest', additional_kwargs={}, example=False))], [ChatGeneration(text='PillowPerfect', generation_info=None, message=AIMessage(content='PillowPerfect', additional_kwargs={}, example=False))], [ChatGeneration(text='FrothMaster', generation_info=None, message=AIMessage(content='FrothMaster', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 116, 'completion_tokens': 19, 'total_tokens': 135}, 'model_name': 'gpt-3.5-turbo'})

Note: you can use OpenAI via your Azure deployment like so

```python
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
deployment_name="your-azure-deployment",
model_name="text-davinci-003"
)
```

# SimpleSequentialChain

The simplest form of sequential chains, where each step has a singular input/output, and the output of one step is the input to the next.

In [12]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    'What is the best name to describe a company that sells {product}?',
)
first_chain = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = ChatPromptTemplate.from_template(
    'Write a 20 word description for the following company: {company_name}',
)
second_chain = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(
    chains=[first_chain, second_chain],
    verbose=True,
)

for product in df.Product.iloc[:5]:
    overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Comfort Beddings
Royal Comfort Beddings is a luxury bedding brand offering top-quality, stylish bedding sets and accessories for a blissful sleep experience.

> Finished chain.


> Entering new SimpleSequentialChain chain...
AquaGuard Phone Protection
AquaGuard Phone Protection offers high-quality waterproof cases and screen protectors to ensure maximum protection for your valuable smartphones.

> Finished chain.


> Entering new SimpleSequentialChain chain...
Opulent Rest
Opulent Rest offers luxurious and high-quality bedding products that provide the ultimate comfort and relaxation for a restful night's sleep.

> Finished chain.


> Entering new SimpleSequentialChain chain...
PillowPro
PillowPro offers premium and customizable pillows designed for optimal comfort and support, ensuring a restful night's sleep.

> Finished chain.


> Entering new SimpleSequentialChain chain...
FrothMaster
FrothMaster is a leading manufacturer of hig

# SequentialChain

A more general form of sequential chains, allowing for multiple inputs/outputs.

In [ ]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    'Translate the following review to English: {review}',
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key='english_review')

second_prompt = ChatPromptTemplate.from_template(
    'Summarize the following review in 1 sentence: {english_review}',
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key='summary')

third_prompt = ChatPromptTemplate.from_template(
    'What language is the following review: {review}'
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key='language')

fourth_prompt = ChatPromptTemplate.from_template(
    'What a follow up response to the following summary in the specified'
    'language: Summary: {summary}\n\n Language: {language}'
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key='follow_up_response')

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=['review'],
    output_variables=['english_review', 'summary', 'follow_up_response'],
    verbose=True,
)